In [1]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/main')

Mounted at /content/gdrive


In [2]:
import tensorflow as tf
import time

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.0-rc3
GPU Enabled: False


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

def parse_fn(text):
  parse = lambda text: [[params['char2idx'].get(c, len(params['char2idx'])) for c in list(text)]]
  return {'words': parse(text)}

In [4]:
params = {'export_dir': '../model/gru_seq2seq_small_export',}

params['char2idx'] = get_vocab('../vocab/char.txt')

params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

subdirs = [x for x in Path(params['export_dir']).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])

predict_fn = tf.contrib.predictor.from_saved_model(latest)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../model/gru_seq2seq_small_export/1570867379/variables/variables


In [5]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  predictions = predict_fn(parse_fn(text_inp))
  print('Output Shape:', predictions['output'].shape)
  for j in range(5):
    print('A{}:'.format(j+1), ' '.join([params['idx2char'].get(idx, len(params['idx2char'])) for idx in predictions['output'][0, :, j]]).replace('<end>', ''))
  print('Spent %.2f sec' % (time.time() - t0))
  print()

Input:你好呀
Output Shape: (1, 7, 5)
A1: 你 好 啊    
A2: 你 好     
A3: 我 也 好    
A4: 你 好 呀    
A5: 我 很 好    
Spent 0.29 sec

Input:你是谁
Output Shape: (1, 16, 5)
A1: 我 是 小 通            
A2: 我 是 你 老 婆 ！          
A3: 我 是 宇 宙 无 敌 小 通        
A4: 我 是 你 的 小 通          
A5: 我 是 你 老 公           
Spent 0.20 sec

Input:今天天气如何
Output Shape: (1, 23, 5)
A1: 消 消 气                    
A2: 天 气 预 报                   
A3: 天 气 预 报 说 。                 
A4: 天 气 预 报 很 好                 
A5: 消 消 气 ， 天 气 预 报 。              
Spent 0.21 sec

Input:天气
Output Shape: (1, 13, 5)
A1: 天 气 预 报         
A2: 天 气 很 好         
A3: 天 气 就 好         
A4: 天 气 不 好         
A5: 天 气 天 气         
Spent 0.11 sec

Input:我感冒了
Output Shape: (1, 9, 5)
A1: 多 喝 水      
A2: 吃 药       
A3: 好 好 照 顾 自 己   
A4: 多 喝 水 ， 多 休 息 哦 
A5: 好 好 照 顾 自 己 哦  
Spent 0.08 sec

Input:你认识刘翔吗
Output Shape: (1, 14, 5)
A1: 当 然 啊 是 啊 ！        
A2: 当 然 啦   我 是 帅 哥      
A3: 当 然 啦 ！ 我 是 帅 哥 。     
A4: 当 然 啦 ！ 我 是 帅 哥 ！     
A5: 认 识 啊 ， 他 就 是 个 大 帅 哥   
Spent 0.12 sec



KeyboardInterrupt: ignored